# IEOR 4703 -- Monte Carlo Simulation for FE

### To calculate $ P(L>C) $ where $L=\sum Y_i e_i$ with $e_i$ exposure to $i^{th}$ credit and $Y_i$ is default indicator w/ correponding default probability of $p_i$

### $P(L > C) = E( 1_{\{L>C\}} )$

### We assume exposures, $e_i$, are independent

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import scipy as sp
from time import time

## Data

In [ ]:
# data
# exposures (in millions) and correponding default probabilities

exposures = np.array([15,    18,    19,   21,   13,   12,    7,    9,   17,  13,    14,    7,   15,   11,    4,    5])
defaultP  = np.array([0.10, 0.20, 0.10, 0.05, 0.15, 0.13, 0.10, 0.08, 0.06, 0.07, 0.05, 0.05, 0.02, 0.04, 0.09, 0.12])

totalExposure = np.sum(exposures)

expectedLoss_exact = np.sum(defaultP*exposures)

# threshold
C = 100

print("total exposure = ", totalExposure)
print("expectedLoss=", expectedLoss_exact)
print("threshold C = ", C)

total exposure =  200
expectedLoss= 17.66
threshold C =  100


## Standard/naive simulation (i.e. not utilizing tilted exponential)

### For the case that C is close to $E(L)$ should work well, however as $C$ gets much larger than $E(L)$ it wouldn't

In [ ]:
np.random.seed(1293561)

m = len(defaultP)
N = 20000
countP = 0

for i in range(N):

    U = np.random.rand(m) # U~Uniform(0,1)

    indicatorP = U<defaultP
    lossP = np.sum(indicatorP*exposures)
    if lossP>C:
        countP = countP+1


# P(L > C) = E( indicator(L>C) )
probP_L_greater_than_C = countP/N
print("P(L>C) =", probP_L_greater_than_C)

P(L>C) = 0.0


### Utilizing tilted exponential for large C, to assess it against standard simulation
#### We calculate both of them simulataneously
#### Note that for tilted exponetial, we need to find optimal value for hyperparameter $t$ for given $C$

# we use tilted probability for the case that C >> E(L)
# there are many methods to find t, we use a proxy bisection method

# try different hyperparameters to assess



In [ ]:
for t in [0, 0.17, 0.172, 0.173]:
    defaultQ = (defaultP*np.exp(t*exposures)) / (defaultP*np.exp(t*exposures)+(1-defaultP))
    C_tilde = np.sum(defaultQ*exposures)
    print ("C_tilde=", C_tilde, "for t=", t, " (threshold=", C,")")

C_tilde= 17.66 for t= 0  (threshold= 100 )
C_tilde= 98.34082267841521 for t= 0.17  (threshold= 100 )
C_tilde= 99.53362055364377 for t= 0.172  (threshold= 100 )
C_tilde= 100.12794680002106 for t= 0.173  (threshold= 100 )


In [ ]:
# from the above biscection
t = 2
defaultQ = (defaultP*np.exp(t*exposures)) / (defaultP*np.exp(t*exposures)+(1-defaultP))

In [ ]:
#np.random.seed(1293561)
np.random.seed(14133561)
m = len(defaultP)
N = 500000
countP = 0
countQ = 0


for i in range(N):

    U = np.random.rand(m) # U~Uniform(0,1)

    indicatorP = U<defaultP
    lossP = np.sum(indicatorP*exposures)

    if lossP>C:
        countP = countP+1


    indicatorQ = U<defaultQ
    lossQ = np.sum(indicatorQ*exposures)
    #
    # pay attention to the way likelihood is being calculated
    numer   = np.prod(np.power(defaultP,indicatorQ)*np.power(1-defaultP,1-indicatorQ))
    denom = np.prod(np.power(defaultQ,indicatorQ)*np.power(1-defaultQ,1-indicatorQ))
    likelihood = numer/denom
    #
    if lossQ>C:
        # pay attention to this part -- very important
        countQ = countQ + 1 * likelihood

# P(L > C) = E( indicator(L>C) )
probP_L_greater_than_C = countP/N
probQ_L_greater_than_C = countQ/N

print("P_P(L>C) =", probP_L_greater_than_C, ", P_Q(L>C) =", probQ_L_greater_than_C)

P_P(L>C) = 2.4e-05 , P_Q(L>C) = 2.711170539019983e-16
